In [67]:
import numpy as np
import pandas as pd
import math as mth
import matplotlib.pyplot as plt
import os.path
from matplotlib.pyplot import figure
import matplotlib.ticker as plticker
from matplotlib.pyplot import figure

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from numpy import mean
from numpy import absolute

In [68]:
totalSubs = 67 
edaSamplingRate = 4
numSeconds = 3

## Make HR and STD and Add Segment Numbers

In [69]:
#Get standard deviantion of each segment and use that as a feature 

for subN in range(1,totalSubs+1):
    hrDir = f'+BCS Experiment Controlled Route Processed/S{subN}/E4/HR.xlsx'
    if os.path.isfile(hrDir):
        heartDF = pd.read_excel(hrDir)

        segmentList = []
        totalCount = 0
        for i in range(0,len(heartDF)):
            tenCount = 0
            while tenCount <numSeconds and totalCount<len(heartDF): #while the heartDF cell is not null:
                segmentList.append(i)
                tenCount = tenCount +1 
                totalCount = totalCount +1
    heartDF['SegmentNum'] = segmentList

    #Normalize 
    column = 'HR'
    heartDF[column] = (heartDF[column] - heartDF[column].min()) / (heartDF[column].max() - heartDF[column].min())
    
    #Add column for standard deviation 
    heartDF['Std'] = heartDF.groupby('SegmentNum')['HR'].transform('std')
    
    heartDF.to_excel(f'./NoMissingNoGPS/addSegNums/S{subN}_HR.xlsx')  
heartDF

,Time,HR,SegmentNum,Std
0,0.55,0.409077,0,0.005354
1,1.55,0.414726,0,0.005354
2,2.55,0.419780,0,0.005354
3,3.55,0.425131,1,0.003766
4,4.55,0.428897,1,0.003766
...,...,...,...,...
955,955.55,0.298583,318,0.004559
956,956.55,0.303042,318,0.004559
957,957.55,0.307997,319,0.001884
958,958.55,0.306213,319,0.001884


### Average Rows with Same Segment Numbers 

In [70]:
for subN in range(1,totalSubs+1):
    final_heartDF = pd.DataFrame()
    hrDir = f'./NoMissingNoGPS/addSegNums/S{subN}_HR.xlsx'
    if os.path.isfile(hrDir):
        avg_heartDF = pd.read_excel(hrDir)
        segAvgHR_List = []
        segAvgHR_List.extend(avg_heartDF.groupby('SegmentNum')['HR'].mean())
        segAvgStd_List = []
        segAvgStd_List.extend(avg_heartDF.groupby('SegmentNum')['Std'].mean())

        del avg_heartDF['Unnamed: 0']
        final_heartDF['AvgHR'] = segAvgHR_List
        final_heartDF['Std'] = segAvgStd_List

        #avg_heartDF.head()
        #final_heartDF['Segment'] = final_heartDF.index
        final_heartDF.insert(loc = 0, column = 'Segment', value = final_heartDF.index)
        final_heartDF.to_excel(f'./NoMissingNoGPS/avgSegNums/S{subN}_avgHR.xlsx') 

## Make EDA and Add Segment Numbers

In [71]:
for subN in range(1,totalSubs+1):
    edaDir = f'+BCS Experiment Controlled Route Processed/S{subN}/E4/EDA.xlsx'
    if os.path.isfile(edaDir):
        edaDF = pd.read_excel(edaDir)

        segmentList = []
        totalCount = 0
        for i in range(0,len(edaDF)):
            tenCount = 0
            while tenCount <(numSeconds*edaSamplingRate) and totalCount<len(edaDF): #while the edaDF cell is not null:
                segmentList.append(i)
                tenCount = tenCount +1 
                totalCount = totalCount +1
    edaDF['SegmentNum'] = segmentList
    column = 'EDA'
    edaDF[column] = (edaDF[column] - edaDF[column].min()) / (edaDF[column].max() - edaDF[column].min())
    
    #Add column for standard deviation 
    edaDF['Std'] = edaDF.groupby('SegmentNum')['EDA'].transform('std')
    
    edaDF.to_excel(f'./NoMissingNoGPS/addSegNums/S{subN}_EDA.xlsx') 

### Average Rows with Same Segment Numbers 

In [72]:
for subN in range(1,totalSubs+1):
    final_edaDF = pd.DataFrame()
    edaDir = f'./NoMissingNoGPS/addSegNums/S{subN}_EDA.xlsx'
    if os.path.isfile(edaDir):
        avg_edaDF = pd.read_excel(edaDir)
        segAvgEDA_List = []
        segAvgEDA_List.extend(avg_edaDF.groupby('SegmentNum')['EDA'].mean())
        
        segAvgStd_List = []
        segAvgStd_List.extend(avg_edaDF.groupby('SegmentNum')['Std'].mean())

        del avg_edaDF['Unnamed: 0']
        final_edaDF['AvgEDA'] = segAvgEDA_List
        final_edaDF['Std'] = segAvgStd_List
        
        final_edaDF.insert(loc = 0, column = 'Segment', value = final_edaDF.index)
        
        
        final_edaDF.to_excel(f'./NoMissingNoGPS/avgSegNums/S{subN}_avgEDA.xlsx') 

In [73]:
final_edaDF

,Segment,AvgEDA,Std
0,0,0.062332,0.011106
1,1,0.038285,0.003749
2,2,0.046618,0.007558
3,3,0.024248,0.004021
4,4,0.021992,0.007520
...,...,...,...
315,315,0.661165,0.014171
316,316,0.681584,0.009972
317,317,0.675720,0.016388
318,318,0.680252,0.005652


## Make Smooth EDA and Add Segment Numbers

In [74]:
for subN in range(1,totalSubs+1):
    edaDir = f'+BCS Experiment Controlled Route Processed/S{subN}/E4/EDA.xlsx'
    if os.path.isfile(edaDir):
        edaDF = pd.read_excel(edaDir)

        segmentList = []
        totalCount = 0
        for i in range(0,len(edaDF)):
            tenCount = 0
            while tenCount <(numSeconds*edaSamplingRate) and totalCount<len(edaDF): #while the edaDF cell is not null:
                segmentList.append(i)
                tenCount = tenCount +1 
                totalCount = totalCount +1
    edaDF['SegmentNum'] = segmentList
    column = 'EDA'
    edaDF[column] = (edaDF[column] - edaDF[column].min()) / (edaDF[column].max() - edaDF[column].min())
    
    #Add column for standard deviation 
    edaDF['Std'] = edaDF.groupby('SegmentNum')['EDA'].transform('std')
    
    edaDF.to_excel(f'./NoMissingNoGPS/addSegNums/S{subN}_EDA.xlsx') 

In [75]:
for subN in range(1,totalSubs+1):
    final_edaDF = pd.DataFrame()
    edaDir = f'./NoMissingNoGPS/addSegNums/S{subN}_EDA.xlsx'
    if os.path.isfile(edaDir):
        avg_edaDF = pd.read_excel(edaDir)
        segAvgEDA_List = []
        segAvgEDA_List.extend(avg_edaDF.groupby('SegmentNum')['EDA'].mean())
        
        segAvgStd_List = []
        segAvgStd_List.extend(avg_edaDF.groupby('SegmentNum')['Std'].mean())

        del avg_edaDF['Unnamed: 0']
        final_edaDF['AvgEDA'] = segAvgEDA_List
        final_edaDF['Std'] = segAvgStd_List
        
        final_edaDF.insert(loc = 0, column = 'Segment', value = final_edaDF.index)
        
        
        final_edaDF.to_excel(f'./NoMissingNoGPS/avgSegNums/S{subN}_avgEDA.xlsx') 

## Make Rating and Add Segment Numbers

In [76]:
for subN in range(1,totalSubs+1):
    rateDir = f'+BCS Experiment Controlled Route Processed/S{subN}/CARMA rating.xlsx'
    if os.path.isfile(rateDir):
        rateDF = pd.read_excel(rateDir)

        segmentList = []
        totalCount = 0
        for i in range(0,len(rateDF)):
            tenCount = 0
            while tenCount < numSeconds and totalCount<len(rateDF): #while the edaDF cell is not null:
                segmentList.append(i)
                tenCount = tenCount +1 
                totalCount = totalCount +1
    rateDF['SegmentNum'] = segmentList
    column = 'Rating'
    rateDF[column] = (rateDF[column] - rateDF[column].min()) / (rateDF[column].max() - rateDF[column].min())
    rateDF.to_excel(f'./NoMissingNoGPS/addSegNums/S{subN}_Rate.xlsx') 

### Average Rows with Same Segment Numbers 

In [77]:
for subN in range(1,totalSubs+1):
    final_rateDF = pd.DataFrame()
    rateDir = f'./NoMissingNoGPS/addSegNums/S{subN}_Rate.xlsx'
    if os.path.isfile(rateDir):
        avg_rateDF = pd.read_excel(rateDir)
        segAvgRate_List = []
        segAvgRate_List.extend(avg_rateDF.groupby('SegmentNum')['Rating'].mean())

        del avg_rateDF['Unnamed: 0']
        final_rateDF['AvgRating'] = segAvgRate_List
        
        final_rateDF.insert(loc = 0, column = 'Segment', value = final_rateDF.index)
        final_rateDF['AvgRating'].values[final_rateDF['AvgRating'].values > 0] = 1
        final_rateDF.to_excel(f'./NoMissingNoGPS/avgSegNums/S{subN}_avgRate.xlsx') 

## Prepare Data

### Concatenate all subjects' HR together, all EDA together, all Rating together

In [78]:
#Concatenate HR
hrDF_full = pd.DataFrame()
for subN in range(1,totalSubs+1):
    hrDir2 = f'./NoMissingNoGPS/avgSegNums/S{subN}_avgHR.xlsx'
    if os.path.isfile(hrDir2):
        heartDF_prep = pd.read_excel(hrDir2)
        heartDF_prep['Sub'] = subN
        data_frames = [hrDF_full,heartDF_prep]
        hrDF_full = pd.concat(data_frames, ignore_index=True)
del hrDF_full['Unnamed: 0']
hrDF_full

,Segment,AvgHR,Std,Sub
0,0,0.641213,0.008270,1
1,1,0.661158,0.002116,1
2,2,0.662782,0.000266,1
3,3,0.659999,0.001184,1
4,4,0.654433,0.002468,1
...,...,...,...,...
21420,315,0.297361,0.010664,67
21421,316,0.272685,0.004821,67
21422,317,0.283619,0.005203,67
21423,318,0.298517,0.004559,67


In [79]:
hrDF_full[450:460]

,Segment,AvgHR,Std,Sub
450,86,0.556099,0.000979,2
451,87,0.554076,0.000493,2
452,88,0.552249,0.000688,2
453,89,0.551074,0.000113,2
454,90,0.550617,0.000000,2
455,91,0.549638,0.000587,2
456,92,0.546962,0.001005,2
457,93,0.545265,0.000113,2
458,94,0.546505,0.000339,2
459,95,0.546701,0.000000,2


In [80]:
#Concatenate EDA
edaDF_full = pd.DataFrame()
for subN in range(1,totalSubs+1):
    edaDir2 = f'./NoMissingNoGPS/avgSegNums/S{subN}_avgEDA.xlsx'
    if os.path.isfile(edaDir2):
        edaDF_prep = pd.read_excel(edaDir2)
        data_frames = [edaDF_full,edaDF_prep]
        edaDF_full = pd.concat(data_frames, ignore_index=True)
del edaDF_full['Unnamed: 0']
edaDF_full

,Segment,AvgEDA,Std
0,0,0.020240,0.004117
1,1,0.016336,0.004148
2,2,0.012835,0.001571
3,3,0.010816,0.002336
4,4,0.027151,0.007642
...,...,...,...
21420,315,0.661165,0.014171
21421,316,0.681584,0.009972
21422,317,0.675720,0.016388
21423,318,0.680252,0.005652


In [81]:
#Concatenate Ratings
rateDF_full = pd.DataFrame()
for subN in range(1,totalSubs+1):
    rateDir2 = f'./NoMissingNoGPS/avgSegNums/S{subN}_avgRate.xlsx'
    if os.path.isfile(rateDir2):
        rateDF_prep = pd.read_excel(rateDir2)
        
        #If heartRate and Rating are off by one...
        hrDir2 = f'./NoMissingNoGPS/avgSegNums/S{subN}_avgHR.xlsx'
        heartDF_prep = pd.read_excel(hrDir2)
        if len(rateDF_prep) > len(heartDF_prep):
            rateDF_prep.drop(rateDF_prep.tail(1).index,inplace=True) # drop last n rows
        
        data_frames = [rateDF_full,rateDF_prep]
        rateDF_full = pd.concat(data_frames, ignore_index=True)
        
        
del rateDF_full['Unnamed: 0']
rateDF_full

,Segment,AvgRating
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
21420,315,0
21421,316,0
21422,317,0
21423,318,0


### Concatenate the concatenated HR, EDA, and Rating dataframes together to create one big dataframe

In [82]:
HER_DF = pd.DataFrame()
HER_DF['Sub'] = hrDF_full['Sub']
HER_DF['HR'] = hrDF_full['AvgHR']
HER_DF['Std_HR'] = hrDF_full['Std']
HER_DF['EDA'] = edaDF_full['AvgEDA']
HER_DF['Std_EDA'] = edaDF_full['Std']
HER_DF['Rate'] = rateDF_full['AvgRating']
HER_DF

,Sub,HR,Std_HR,EDA,Std_EDA,Rate
0,1,0.641213,0.008270,0.020240,0.004117,0
1,1,0.661158,0.002116,0.016336,0.004148,0
2,1,0.662782,0.000266,0.012835,0.001571,0
3,1,0.659999,0.001184,0.010816,0.002336,0
4,1,0.654433,0.002468,0.027151,0.007642,0
...,...,...,...,...,...,...
21420,67,0.297361,0.010664,0.661165,0.014171,0
21421,67,0.272685,0.004821,0.681584,0.009972,0
21422,67,0.283619,0.005203,0.675720,0.016388,0
21423,67,0.298517,0.004559,0.680252,0.005652,0


In [83]:
HER_DF.to_csv('./tempERASE.csv')

In [84]:
count_nan = HER_DF['Std_HR'].isnull().sum()
print ('Count of NaN: ' + str(count_nan))

Count of NaN: 15


In [85]:
HER_DF=HER_DF.dropna(axis=0)

# Reset index after drop
HER_DF=HER_DF.dropna().reset_index(drop=True)

In [86]:
def confusionMatrix(x, y):
    if len(x) != len(y):
        print("incorrect sizes")
        return
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for i in range(len(x)):
        if x[i] == 1 and y[i] == 1:
            tp += 1
        elif x[i] == 0 and y[i] == 1:
            fp += 1
        elif x[i] == 1 and y[i] == 0:
            fn += 1
        else:
            tn += 1
    print(tp, fn)
    print(fp, tn)

## "Classifier"

In [116]:
numTrees = 100
#Prediction target----parameter
y = HER_DF['Rate'].values.tolist()

#List of features
feature_names = ['HR', 'EDA', 'Std_HR', 'Std_EDA']

#X value/features----parameter
X = HER_DF[feature_names]

#Random Forest 
theModel = RandomForestClassifier()

#Split data into training and validation sets
train_X, val_X, train_y, val_y = train_test_split(X, y, shuffle= True)#random_state = 0, shuffle=True)

theModel = RandomForestClassifier(n_estimators = numTrees)
theModel.fit(train_X, train_y)
val_predictions = theModel.predict(val_X)
confusionMatrix(val_y, val_predictions)
print(accuracy_score(val_y, val_predictions))
#print(val_predictions)
#print(val_y)   

#f1 = f1_score(val_y, val_predictions)
#print(f1)

435 1183
253 3482
0.7317392116570147


In [88]:
HER_DF[~HER_DF['Sub'].isin([67])]

,Sub,HR,Std_HR,EDA,Std_EDA,Rate
0,1,0.641213,0.008270,0.020240,0.004117,0
1,1,0.661158,0.002116,0.016336,0.004148,0
2,1,0.662782,0.000266,0.012835,0.001571,0
3,1,0.659999,0.001184,0.010816,0.002336,0
4,1,0.654433,0.002468,0.027151,0.007642,0
...,...,...,...,...,...,...
21085,66,0.862555,0.004852,0.921488,0.014241,1
21086,66,0.848629,0.004523,0.950784,0.006706,1
21087,66,0.835362,0.004361,0.936357,0.009590,1
21088,66,0.821382,0.004934,0.902414,0.015127,1


In [89]:
#Leave one subject out 
accuracies = []
for subject in range(1,totalSubs+1):
    tempDF = HER_DF[~HER_DF['Sub'].isin([subject])]
    exDF = HER_DF[HER_DF['Sub'].isin([subject])]
    feature_names = ['HR', 'EDA', 'Std_HR', 'Std_EDA']
    ex_y = exDF['Rate'].values.tolist()
    ex_X = exDF[feature_names]
    
    y = tempDF['Rate'].values.tolist()
    X = tempDF[feature_names]
    
    
    #Random Forest 
    theModel = RandomForestClassifier()

    #Split data into training and validation sets
    
    train_X = X
    train_y = y
    val_X = ex_X
    val_y = ex_y
    
    theModel = RandomForestClassifier(n_estimators = numTrees)
    theModel.fit(train_X, train_y)
    val_predictions = theModel.predict(val_X)
    accuracies.append(accuracy_score(val_y, val_predictions))
    print(accuracy_score(val_y, val_predictions))
print(accuracies)    

0.699724517906336
1.0
1.0
0.6910299003322259
0.7263843648208469
0.8970588235294118
0.6970802919708029
0.5814889336016097
0.7631578947368421
0.7241379310344828
0.6640625
0.5085227272727273
0.6263345195729537
0.5
0.6474358974358975
0.6534216335540839
0.22818791946308725
0.8133333333333334
0.6260869565217392
0.4716981132075472
0.7577464788732394
0.6577181208053692
0.3681159420289855
0.5981012658227848
0.4668769716088328
0.7183098591549296
0.8125
0.78125
0.7465277777777778
0.7602523659305994
0.6034985422740525
0.7625
0.7805755395683454
0.5189873417721519
0.8122605363984674
0.7324414715719063
0.6722689075630253
0.7083333333333334
0.6201298701298701
0.029900332225913623
0.7909090909090909
0.6239782016348774
0.8143236074270557
0.496875
0.7298701298701299
0.625
0.6175548589341693
0.5654952076677316
0.815625
0.5398773006134969
0.6072507552870091
0.8288288288288288
0.5098039215686274
0.6993670886075949
0.5704918032786885
0.631578947368421
0.6056910569105691
0.6273291925465838
0.6840659340659341


In [113]:
counts = HER_DF['Sub'].value_counts()
counts = counts.sort_index()
countsList = counts.values.tolist()

total = 0
for i in range(len(countsList)):
    total += countsList[i]*accuracies[i]
avg = total / np.sum(countsList)
print(avg)
print(np.sum(accuracies)/len(accuracies))

0.6572162540868753
0.6608546195240684


In [90]:
print("Chance Accuracy: ")
count0 = (HER_DF['Rate'] == 0).sum()
print(count0)
count1 = (HER_DF['Rate'] == 1).sum()
print(count1)
print((count0/(count1+count0))*100, '%')

Chance Accuracy: 
15170
6240
70.85474077533863 %


In [117]:
from sklearn.metrics import f1_score
#f1_score(val_y, val_predictions, average=None)
f1_score(val_y, val_predictions)


0.37727666955767564